In [192]:
import pyscheduling.SMSP.risijwiTi as smsp_risijT
from pyscheduling.SMSP.SingleMachine import Job

from docplex.cp.model import CpoModel
from docplex.cp.solver.cpo_callback import CpoCallback
from docplex.cp.expression import INTERVAL_MAX

In [193]:
def csp_transform_solution(msol, E_i, instance, objective):

    sol = instance.create_solution()
    k_tasks = []
    for i in range(instance.n):
        start = msol[E_i[i]][0]
        end = msol[E_i[i]][1]
        k_tasks.append(Job(i,start,end))

        k_tasks = sorted(k_tasks, key= lambda x: x[1])
        sol.machine.job_schedule = k_tasks

    if objective == "wiCi":
        sol.wiCi()
    elif objective == "wiTi":
        sol.wiTi()
    return sol

In [208]:
instance = smsp_risijT.risijwiTi_Instance.generate_random(15,Wmax=4)
E = range(instance.n)

# Build transition matrix if setup is included
trans_matrix = None
if hasattr(instance, 'S'):
    trans_matrix = [0 for i in range(instance.n + 1)
                for j in range(instance.n + 1)]
    for i in range(instance.n):
        # Setup of the first job
        trans_matrix[i+1] = instance.S[i][i]
        for j in range(instance.n):
            if i != j:
                # Setup between i and j
                trans_matrix[(i+1)*(instance.n+1) + j +
                         1] = instance.S[i][j]

In [209]:
# Construct the model
model = CpoModel("smspModel")
objective = "wiTi"

# Jobs interval_vars including the release date and processing times constraints
E_i, S_i, iv_array, types_array = [], [], [], []
for i in E:
    start_period = (instance.R[i], INTERVAL_MAX) if hasattr(instance, 'R') else (0, INTERVAL_MAX)
    job_i = model.interval_var( start = start_period,
                                size = instance.P[i], optional= False, name=f'E[{i}]')
    E_i.append(job_i)
    
    # Arrays for sequence variable
    iv_array.append(job_i)
    types_array.append(i)
    
    # Add setup task if setup is included
    if hasattr(instance, 'S'):
        # Setup can only start after the release of job_i
        setup_task = model.interval_var(start=start_period,
                                        optional=False, name=f'S[{i}]')
        S_i.append(setup_task)
        
        # Arrays for sequence variable
        iv_array.append(setup_task)
        types_array.append(0)
        
        # Processing of job i starts right after its setup
        model.add((model.end_at_start(setup_task, job_i)
                               ).set_name(f'SP[{i}]'))
        
# Sequential execution on the machine
machine_sequence = model.sequence_var( iv_array, types_array )
model.add( model.no_overlap(machine_sequence) )

# Add the setup constraint
if hasattr(instance, 'S'):
    for i in E:
        # Setup time size
        dep = S_i[i]

        model.add((model.size_of(dep) ==
                   model.element(trans_matrix,
                                 (model.type_of_prev(
                       machine_sequence, dep, -1, 0) + 1) * (instance.n+1)
            + i + 1)).set_name(f'Dep[{i},{j}]')
        )

# Define the objective 
if objective == "wiCi":
    model.add(model.minimize( sum( instance.W[i] * model.end_of(E_i[i]) for i in E ) )) # sum_{i in E} wi * ci
elif objective == "cmax":
    model.add(model.minimize( max( model.end_of(E_i[i]) for i in E ) )) # max_{i in E} ci 
elif objective == "wiTi":
    model.add( model.minimize( 
        sum( instance.W[i] * model.max(model.end_of(E_i[i]) - instance.D[i], 0) for i in E ) # sum_{i in E} wi * Ti
    ))

In [210]:
msol = model.solve(LogVerbosity="Normal", Workers=1, TimeLimit=30, LogPeriod=1000000,
                   log_output=True, trace_log=True, add_log_to_solution=True, RelativeOptimalityTolerance=0)

 ! --------------------------------------------------- CP Optimizer 20.1.0.0 --
 ! Minimization problem - 31 variables, 31 constraints
 ! TimeLimit            = 30
 ! Workers              = 1
 ! LogPeriod            = 1000000
 ! RelativeOptimalityTolerance = 0
 ! Initial process time : 0.02s (0.02s extraction + 0.00s propagation)
 !  . Log search space  : 220.8 (before), 220.8 (after)
 !  . Memory usage      : 574.9 kB (before), 574.9 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed            Branch decision
                        0         31                 -
 + New bound is 0
 *         28868    38771  0.25s               (gap is 100.0%)
 *         26008    58069  0.38s               (gap is 100.0%)
 *         24602    61709  0.43s               (gap is 100.0%)
 *         20818    62514  0.45s               (gap is 100.0%)
 *         19468    65318  0.50s               (gap i

In [167]:
print("\n".join(map(str,msol[machine_sequence][::])))

S[3]: (start=35, end=35, size=0, length=0)
E[3]: (start=35, end=41, size=6, length=6)
S[2]: (start=42, end=49, size=7, length=7)
E[2]: (start=49, end=57, size=8, length=8)
S[1]: (start=57, end=66, size=9, length=9)
E[1]: (start=66, end=135, size=69, length=69)
S[5]: (start=135, end=216, size=81, length=81)
E[5]: (start=216, end=281, size=65, length=65)
S[6]: (start=281, end=325, size=44, length=44)
E[6]: (start=325, end=362, size=37, length=37)
S[4]: (start=362, end=402, size=40, length=40)
E[4]: (start=402, end=435, size=33, length=33)
S[0]: (start=435, end=474, size=39, length=39)
E[0]: (start=474, end=508, size=34, length=34)


In [185]:
instance.P

[34, 69, 8, 6, 33, 65, 37]

In [183]:
instance.R

[218, 40, 42, 33, 177, 57, 211]

In [191]:
instance.D

[309, 196, 117, 61, 328, 236, 361]

In [186]:
i,j = ( 2, 1 )
print("IV- Setup:", msol[S_i[j]] , "Proc:", msol[E_i[j]])
print("Trans_matrix: ",trans_matrix[(i+1) * (instance.n+1) + j + 1])
print(f"S[{i}, {j}] = {instance.S[i][j]}")

IV- Setup: IntervalVarValue(start=57, end=66, size=9) Proc: IntervalVarValue(start=66, end=135, size=69)
Trans_matrix:  9
S[2, 1] = 9


In [212]:
sol = csp_transform_solution(msol, E_i, instance, objective)

In [213]:
sol.wiTi()
print(sol.machine.total_weighted_lateness(sol.instance))

17363


In [190]:
instance.to_txt("example_instance.txt")

In [189]:
sol.to_txt("example_sol.txt")

In [169]:
instance.D

[309, 196, 117, 61, 328, 236, 361]

In [211]:
solve_result = smsp_risijT.Heuristics.ACTS_WSECi(instance)
print(solve_result)

Search stopped with status : FEASIBLE
 Solution is : 
 Cmax : 8180
Job_schedule (job_id , start_time , completion_time) | Completion_time
(12, 32, 41) : (6, 41, 70) : (10, 70, 92) : (2, 92, 146) : (8, 146, 251) : (11, 251, 365) : (0, 365, 506) : (3, 506, 589) : (13, 589, 638) : (14, 638, 853) : (7, 853, 1079) : (5, 1079, 1302) : (9, 1302, 1447) : (1, 1447, 1629) : (4, 1629, 1859) | 8180 
Runtime is : 0.00036701600765809417s 
time to best is : -1s 

